In [26]:
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from PIL import Image
from AB3DMOT_libs.kitti_utils import read_label, compute_box_3d, draw_projected_box3d, Calibration, Object3d

import pdb

# Visualize detection result
Load detection results from "data/KITTI/result_sha/seq" and images from "data_root/image_02/seq". Save visualization results in "./results/result_sha/det_image_vis/seq" 

In [49]:
def visdet(result_sha, data_root, seq):
    width = 1242
    height = 374
    def show_image_with_boxes(img, objects_res, object_gt, calib, save_path, height_threshold=0):
        print("Entering show_image_with_boxes()")
        img2 = np.copy(img) 
        # for each object, compute the bouding box in 2D image. Use the same color for objects of same obj.id. 
        print(img2.shape)
        print(len(objects_res))
        for obj in objects_res:
            # print(obj)
            box3d_pts_2d, _ = compute_box_3d(obj, calib.P) # calib.P is the projection matrix from camera coord to image coord
            color_tmp = tuple([255,0,0])
            img2 = draw_projected_box3d(img2, box3d_pts_2d, color=color_tmp)
            text = 'ID: %d' % obj.id
            if box3d_pts_2d is not None:
                print(box3d_pts_2d.shape)
                print(box3d_pts_2d)
                img2 = cv2.putText(img2, text, (int(box3d_pts_2d[4, 0]), int(box3d_pts_2d[4, 1]) - 8), cv2.FONT_HERSHEY_TRIPLEX, 0.5, color=color_tmp)
            else:
                print("It is None!!")

        img = Image.fromarray(img2)
        img = img.resize((width, height))
        img.save(save_path)
        print("Done with this frame! Exiting show_image_with_boxes()")
    image_dir = os.path.join(data_root, 'image_2/%s.png' % seq)
    calib_file = os.path.join(data_root, 'calib/%s.txt' % seq) 

    save_3d_bbox_dir = os.path.join('./results', result_sha, 'det_image_vis/%s' % seq)
    print(save_3d_bbox_dir)
    if not os.path.exists(save_3d_bbox_dir):
        os.makedirs(save_3d_bbox_dir)

    # seq_file = os.path.join('data/KITTI', result_sha, "%s.txt"%seq)
    # seq_dets = np.loadtxt(seq_file, delimiter=',')
    seq_file = os.path.join('/team1/codes/3dObjDet/OpenPCDet_Ani/results/3dod/vis/', "seq_%s.csv"%seq)
    seq_dets = np.loadtxt(seq_file, delimiter=' ')
    print(seq_dets.shape)
    if len(seq_dets.shape) == 1: seq_dets = np.expand_dims(seq_dets, axis=0) 	
    if seq_dets.shape[1] == 0:
        return
    calib_tmp = Calibration(calib_file)
    # loop over frame
    min_frame, max_frame = int(seq_dets[:, 0].min()), int(seq_dets[:, 0].max())
    print(min_frame, max_frame)
    # print(seq_dets)
    for frame in range(min_frame, max_frame + 1):
        try:
            # img_tmp = np.array(Image.open(os.path.join(image_dir,"%06d.png"%frame)))
            img_tmp = np.array(Image.open(image_dir))
        except:
            break
        # logging
        print_str = 'processing %s:  %d/%d   \r' % (seq,  frame, max_frame)
        print(print_str)

        dets = seq_dets[seq_dets[:,0] == frame]            # h, w, l, x, y, z, theta in camera coordinate follwing KITTI convention
        print(len(dets))
        objects = []
        for i, det in enumerate(dets):
            print(det)
            #str_to_srite = '%s -1 -1 %f %f %f %f %f %f %f %f %f %f %f %f %f %d\n' % ("Car", det[14],
            #        det[2], det[3], det[4], det[5], 
            #        det[7], det[8], det[9], det[10], det[11], det[12], det[13], det[-1], 1)
            str_to_srite = '%s -1 -1 %f 195.4448 186.4714 438.3607 343.4189 %f %f %f %f %f %f %f %f %d\n' % ("Car", det[14], 
                   det[10], det[11], det[12], det[7], det[8], det[9], det[13], det[-1], 1)
            print(str_to_srite)
            objects.append(Object3d(str_to_srite))
            # pdb.set_trace()
        print(len(objects))
        save_path_tmp = os.path.join(save_3d_bbox_dir, '%06d.jpg' % frame)
        # save_path_tmp = os.path.join(save_3d_bbox_dir, '%')
        show_image_with_boxes(img_tmp, objects, [], calib_tmp, save_path_tmp)

## Example

I put 21 images in 

In [50]:
result_sha = "pointrcnn_Car_test"
# data_root = './data/KITTI/resources/testing'
data_root = "/shared/kitti/training/"
seq = "000008"
visdet(result_sha, data_root, seq)

./results/pointrcnn_Car_test/det_image_vis/000008
(18, 15)
0 0
processing 000008:  0/0   
18
[ 0.          1.          0.          0.          0.          0.
  0.99984503  3.71338177  1.57429433  1.47335505 14.72933674 -1.06972837
 -0.78867513  5.9638257   0.        ]
Car -1 -1 0.000000 195.4448 186.4714 438.3607 343.4189 14.729337 -1.069728 -0.788675 3.713382 1.574294 1.473355 5.963826 0.000000 1

[ 0.          1.          0.          0.          0.          0.
  0.9997862   3.64700532  1.55678117  1.5720284   8.15506172  1.22846735
 -0.83391374  2.82995129  0.        ]
Car -1 -1 0.000000 195.4448 186.4714 438.3607 343.4189 8.155062 1.228467 -0.833914 3.647005 1.556781 1.572028 2.829951 0.000000 1

[ 0.          1.          0.          0.          0.          0.
  0.99798751  3.34747529  1.61376679  1.58728766  3.98584628  2.6945138
 -0.86719096  6.00238419  0.        ]
Car -1 -1 0.000000 195.4448 186.4714 438.3607 343.4189 3.985846 2.694514 -0.867191 3.347475 1.613767 1.587288 6.0023